In [4]:
# dependencies
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import create_engine, func
from sqlalchemy import create_engine, inspect

In [5]:
phytoplankton = pd.read_csv("data/phytoplankton.csv")
zooplankton =  pd.read_csv("data/zooplankton.csv")
phytoplankton_color = pd.read_csv("data/phytoplankton_ci.csv")

In [6]:
phytoplankton.head()

,FID,id,Route,Latitude,Longitude,SampleDateUTC,Year,Month,Day,Time_24hr,...,caab_code,taxon_start_date,taxon_per_m3,phyto_comments,acknowledgements,first_occurrence,parent_taxon_name,training,comments,geom
0,soop_auscpr_phyto_flat_map.fid--404dafa0_17a9a...,30880720,ABAB,-37.9138,135.3310,2020-02-27T02:01:35Z,2020,2,27,02:01:35,...,50566000.0,NaN,3333.3,NaN,Wallenius Wilhelmsen,NaN,NaN,NaN,NaN,POINT (135.33099 -37.91378)
1,soop_auscpr_phyto_flat_map.fid--404dafa0_17a9a...,30880722,ABAB,-37.9138,135.3310,2020-02-27T02:01:35Z,2020,2,27,02:01:35,...,52030054.0,NaN,666.7,*Noted as present in the zoop count*,Wallenius Wilhelmsen,NaN,NaN,NaN,NaN,POINT (135.33099 -37.91378)
2,soop_auscpr_phyto_flat_map.fid--404dafa0_17a9a...,30880721,ABAB,-37.9138,135.3310,2020-02-27T02:01:35Z,2020,2,27,02:01:35,...,52030900.0,2011-05-17,666.7,*Noted as present in the zoop count*,Wallenius Wilhelmsen,2011-09-20,Tripos spp.,"Internal training Pru Bonhma, Gustaaf Hallegraeff",NaN,POINT (135.33099 -37.91378)
3,soop_auscpr_phyto_flat_map.fid--404dafa0_17a9a...,30880726,ABAB,-37.8766,134.9125,2020-02-27T03:10:21Z,2020,2,27,03:10:21,...,99530018.0,2011-05-17,6666.7,NaN,Wallenius Wilhelmsen,2011-09-15,Pseudo-nitzschia spp.,"Internal training Pru Bonhma, Gustaaf Hallegraeff",NaN,POINT (134.91247 -37.87656)
4,soop_auscpr_phyto_flat_map.fid--404dafa0_17a9a...,30880727,ABAB,-37.8766,134.9125,2020-02-27T03:10:21Z,2020,2,27,03:10:21,...,NaN,NaN,3333.3,NaN,Wallenius Wilhelmsen,NaN,NaN,NaN,NaN,POINT (134.91247 -37.87656)


In [7]:
#drop unneeded columns for phytoplankton
phytoplankton = phytoplankton.drop(["Route",
                                    "Time_24hr",
                                    "FID",
                                    "family",
                                    "genus",
                                    "species",
                                    "caab_code",
                                   "taxon_start_date",
                                    "taxon_per_m3",
                                   "phyto_comments",
                                   "acknowledgements",
                                   "first_occurrence",
                                   "parent_taxon_name",
                                   "training",
                                   "comments",
                                    "geom"
                                  ],
                                  axis=1)

In [8]:
phytoplankton.head()

,id,Latitude,Longitude,SampleDateUTC,Year,Month,Day,taxon_name,taxon_group
0,30880720,-37.9138,135.3310,2020-02-27T02:01:35Z,2020,2,27,Steenstrupiella spp.,Ciliate
1,30880722,-37.9138,135.3310,2020-02-27T02:01:35Z,2020,2,27,Tripos trichoceros,Dinoflagellate
2,30880721,-37.9138,135.3310,2020-02-27T02:01:35Z,2020,2,27,Tripos furca type,Dinoflagellate
3,30880726,-37.8766,134.9125,2020-02-27T03:10:21Z,2020,2,27,Pseudo-nitzschia seriata complex >3 µm,Pennate diatom
4,30880727,-37.8766,134.9125,2020-02-27T03:10:21Z,2020,2,27,Tripos macroceros type,Dinoflagellate


In [9]:
for col in zooplankton.columns:
    print(col)

FID
id
Route
Latitude
Longitude
SampleDateUTC
Year
Month
Day
Time_24hr
taxon_name
family
genus
species
sex
life_stage
taxon_group
taxon_eco_group
caab_code
aphiaid
taxon_start_date
taxon_per_m3
zoop_comments
acknowledgements
first_occurrence
parent_taxon_name
training
comments
geom


In [11]:
#drop unneeded columns for zooplankton
zooplankton = zooplankton.drop(["FID",
                                "Route",
                                "Time_24hr",
                                "family",
                                "genus",
                                "species",
                                "sex",
                                  "life_stage",
                                    "caab_code",
                                    "aphiaid",  
                                   "taxon_start_date",
                                   "zoop_comments",
                                   "acknowledgements",
                                   "first_occurrence",
                                   "parent_taxon_name",
                                   "training",
                                   "comments",
                                    "taxon_eco_group",
                                "geom"
                                  ],
                                  axis=1)

In [12]:
zooplankton.head()

,id,Latitude,Longitude,SampleDateUTC,Year,Month,Day,taxon_name,taxon_group,taxon_per_m3
0,60974299,-37.9138,135.3310,2020-02-27T02:01:35Z,2020,2,27,Calanoid j,COPEPOD,2.0000
1,60974379,-37.9138,135.3310,2020-02-27T02:01:35Z,2020,2,27,Chaetognatha,NON-COPEPOD,0.6667
2,60974380,-37.9138,135.3310,2020-02-27T02:01:35Z,2020,2,27,Foraminifera,NON-COPEPOD,1.3333
3,60974378,-37.9138,135.3310,2020-02-27T02:01:35Z,2020,2,27,Oithona spp. j,COPEPOD,4.0000
4,60974296,-37.8766,134.9125,2020-02-27T03:10:21Z,2020,2,27,Oithona plumifera f,COPEPOD,0.6667


In [13]:
for col in phytoplankton_color.columns:
    print(col)

FID
measurement
route_code
start_port
end_port
route_frequency
route_start_date
vessel_name
trip_code
colour_index
colour
pci_comments
acknowledgements
TIME
geom
latitude
longitude


In [14]:
#drop unneeded columns for phytoplankton color index
phytoplankton_color = phytoplankton_color.drop(["FID",
                                                "route_code",
                                                "start_port",
                                                "end_port",
                                                "trip_code",
                                                "colour_index",
                                                "measurement",
                                               "acknowledgements",
                                               "route_frequency",
                                               "route_start_date",
                                               "pci_comments",
                                                "geom",
                                                  ],
                                                  axis=1)

In [15]:
phytoplankton_color

,vessel_name,colour,TIME,latitude,longitude
0,Salome IMO,VERY_PALE_GREEN,2020-02-27T02:01:35Z,-37.9138,135.3310
1,Salome IMO,VERY_PALE_GREEN,2020-02-27T02:18:47Z,-37.9046,135.2263
2,Salome IMO,VERY_PALE_GREEN,2020-02-27T02:35:58Z,-37.8954,135.1217
3,Salome IMO,VERY_PALE_GREEN,2020-02-27T02:53:10Z,-37.8860,135.0171
4,Salome IMO,VERY_PALE_GREEN,2020-02-27T03:10:21Z,-37.8766,134.9125
...,...,...,...,...,...
27627,Kweichow IMO 9070694,VERY_PALE_GREEN,2016-10-17T18:52:05Z,-17.2799,146.1455
27628,Kweichow IMO 9070694,VERY_PALE_GREEN,2016-10-17T19:18:59Z,-17.1994,146.1217
27629,Kweichow IMO 9070694,VERY_PALE_GREEN,2016-10-17T19:45:53Z,-17.1189,146.0979
27630,Kweichow IMO 9070694,VERY_PALE_GREEN,2016-10-17T20:12:47Z,-17.0384,146.0742


In [16]:
# Create an engine for the plankton.sqlite database
engine = create_engine("sqlite:///planktontest.sqlite", echo=False)
conn = engine.connect()

In [12]:
# add csv to data base function
# def read_sql(data):
#     data.to_sql(f"'{data}'", conn, if_exists = "append")


In [13]:
# # add csv to data base
# read_sql(phytoplankton)
# read_sql(zooplankton)
# read_sql(phytoplankton_color)

In [17]:
phytoplankton.to_sql("phytoplankton", conn,)
zooplankton.to_sql("zooplankton", conn,)
phytoplankton_color.to_sql("phytoplankton_color_index", conn,)

In [15]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [16]:
# Reflect Database into ORM classes
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

[]

In [17]:
# Use `engine.execute` to select and display the first 10 rows from the station table
engine.execute('SELECT * FROM zooplankton LIMIT 5').fetchall()

[(0, 'soop_auscpr_zoop_flat_map.fid--620ea8f0_17a9a6b7f95_-53b3', 60974299, 'ABAB', -37.9138, 135.33100000000002, '2020-02-27T02:01:35Z', 2020, 2, 27, '02:01:35', 'Calanoid j', None, None, None, 'COPEPOD', 'POINT (135.33099 -37.91378)'),
 (1, 'soop_auscpr_zoop_flat_map.fid--620ea8f0_17a9a6b7f95_-53b2', 60974379, 'ABAB', -37.9138, 135.33100000000002, '2020-02-27T02:01:35Z', 2020, 2, 27, '02:01:35', 'Chaetognatha', None, None, None, 'NON-COPEPOD', 'POINT (135.33099 -37.91378)'),
 (2, 'soop_auscpr_zoop_flat_map.fid--620ea8f0_17a9a6b7f95_-53b1', 60974380, 'ABAB', -37.9138, 135.33100000000002, '2020-02-27T02:01:35Z', 2020, 2, 27, '02:01:35', 'Foraminifera', None, None, None, 'NON-COPEPOD', 'POINT (135.33099 -37.91378)'),
 (3, 'soop_auscpr_zoop_flat_map.fid--620ea8f0_17a9a6b7f95_-53b0', 60974378, 'ABAB', -37.9138, 135.33100000000002, '2020-02-27T02:01:35Z', 2020, 2, 27, '02:01:35', 'Oithona spp. j', 'Oithonidae', 'Oithona', 'spp.', 'COPEPOD', 'POINT (135.33099 -37.91378)'),
 (4, 'soop_auscpr

In [18]:
# Query phytoplankton Records in the the Database
data = pd.read_sql("SELECT * FROM phytoplankton", conn)

In [19]:
data.head()

,index,id,Route,Latitude,Longitude,SampleDateUTC,Year,Month,Day,Time_24hr,taxon_name,family,genus,species,taxon_group,geom
0,0,30880720,ABAB,-37.9138,135.3310,2020-02-27T02:01:35Z,2020,2,27,02:01:35,Steenstrupiella spp.,Tintinnidae,Steenstrupiella,spp.,Ciliate,POINT (135.33099 -37.91378)
1,1,30880722,ABAB,-37.9138,135.3310,2020-02-27T02:01:35Z,2020,2,27,02:01:35,Tripos trichoceros,Ceratiaceae,Tripos (Tripoceratium),trichoceros,Dinoflagellate,POINT (135.33099 -37.91378)
2,2,30880721,ABAB,-37.9138,135.3310,2020-02-27T02:01:35Z,2020,2,27,02:01:35,Tripos furca type,Ceratiaceae,Tripos (Ceratium),furca type,Dinoflagellate,POINT (135.33099 -37.91378)
3,3,30880726,ABAB,-37.8766,134.9125,2020-02-27T03:10:21Z,2020,2,27,03:10:21,Pseudo-nitzschia seriata complex >3 µm,Bacillariaceae,Pseudo-nitzschia,seriata complex,Pennate diatom,POINT (134.91247 -37.87656)
4,4,30880727,ABAB,-37.8766,134.9125,2020-02-27T03:10:21Z,2020,2,27,03:10:21,Tripos macroceros type,Ceratiaceae,Tripos (Tripoceratium),macroceros type,Dinoflagellate,POINT (134.91247 -37.87656)
